In [1]:
import torch
torch.__version__

'2.5.0+cu121'

In [2]:
import torchvision
from torchvision import models
torchvision.__version__

'0.20.0+cu121'

In [3]:
resnet = models.resnet50(pretrained=True)

# Save the model state_dict
torch.save(resnet.state_dict(), "resnet50_weights.pth")
resnet = models.resnet50()

# Load the state_dict into the model
resnet.load_state_dict(torch.load("resnet50_weights.pth"))

C:\Users\austi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\austi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\austi\AppData\Local\Temp\ipykernel_19276\3845282439.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickl

<All keys matched successfully>

In [4]:
from torch import nn
class EncoderCNN(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(pretrained=True)

        for param in resnet.parameters():
            param.requires_grad_(False)
        
        modules = list(resnet.children())[:-2]
        self.resnet = nn.Sequential(*modules)

    def forward(self, images):
        features = self.resnet(images)                                   
        features = features.permute(0, 2, 3, 1)                          
        features = features.view(features.size(0), -1, features.size(-1)) 
        return features                                   

        return features

In [5]:
enc = EncoderCNN()  # ✅ Create an instance
print(enc.resnet.children)   # ✅ Access resnet


<bound method Module.children of Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [6]:
a=torch.rand((1,3,224,224))

In [7]:
enc(a).shape

torch.Size([1, 49, 2048])

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from transformers import BertTokenizer

class EncoderCNN(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(pretrained=True)
        
        for param in resnet.parameters():
            param.requires_grad_(False)
        
        self.resnet = nn.Sequential(*list(resnet.children())[:-2])

    def forward(self, images):
        features = self.resnet(images)                                     # (batch_size, 2048, 7, 7)
        features = features.permute(0, 2, 3, 1).view(features.size(0), -1, features.size(1)) # (batch, 49, 2048)
        return features

class Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        super().__init__()
        self.encoder_proj = nn.Linear(encoder_dim, attention_dim)
        self.decoder_proj = nn.Linear(decoder_dim, attention_dim)
        self.attention_proj = nn.Linear(attention_dim, 1)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, encoder_out, decoder_hidden):
        encoder_projected = self.encoder_proj(encoder_out)  
        decoder_hidden_projected = self.decoder_proj(decoder_hidden).unsqueeze(1)  
        
        attention = self.relu(encoder_projected + decoder_hidden_projected)
        attention = self.attention_proj(attention).squeeze(2)  
        alpha = self.softmax(attention)  
        
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  
        return attention_weighted_encoding, alpha

class Decoder(nn.Module):
    def __init__(self, encoder_dim, decoder_dim, embed_size, attention_dim, drop):
        super().__init__()
        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)
        self.embedding = nn.Embedding(BertTokenizer.from_pretrained('bert-base-uncased').vocab_size, embed_size)
        self.init_h = nn.Linear(encoder_dim, decoder_dim)
        self.init_c = nn.Linear(encoder_dim, decoder_dim)
        self.lstm_cell = nn.LSTMCell(embed_size + encoder_dim, decoder_dim)
        self.fcn = nn.Linear(decoder_dim, self.embedding.num_embeddings)
        self.drop = nn.Dropout(drop)

    def inithidden(self, encoder_out):
        mean_enc = encoder_out.mean(dim=1)
        h, c = self.init_h(mean_enc), self.init_c(mean_enc)
        return h, c

    def forward(self, features, captions):
        embeds = self.embedding(captions)  
        h, c = self.inithidden(features)
        seq_length, batch_size, num_features = captions.size(1), captions.size(0), features.size(1)

        preds = torch.zeros(batch_size, seq_length, self.embedding.num_embeddings).to(features.device)
        alphas = torch.zeros(batch_size, seq_length, num_features).to(features.device)

        for s in range(seq_length):
            context, alpha = self.attention(features, h)
            lstm_input = torch.cat((embeds[:, s, :], context), dim=1)  # Correct concatenation
            h, c = self.lstm_cell(lstm_input, (h, c))
            output = self.fcn(self.drop(h))
            preds[:, s, :] = output
            alphas[:, s, :] = alpha

        return preds, alphas

class fullmodel(nn.Module):
    def __init__(self, embed_size, decoder_dim, encoder_dim, attention_dim, drop=0.3):
        super().__init__()
        self.encoder = EncoderCNN()
        self.decoder = Decoder(
            embed_size=embed_size,
            attention_dim=attention_dim,
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim,
            drop=drop
        )

    def forward(self, images, captions):
        features = self.encoder(images)
        outputs, alphas = self.decoder(features, captions)
        return outputs, alphas

In [11]:
import torch

# Define model hyperparameters
embed_size = 256
decoder_dim = 512
encoder_dim = 2048
attention_dim = 256

# Instantiate the model
model = fullmodel(embed_size, decoder_dim, encoder_dim, attention_dim)

# Generate sample inputs
batch_size = 2
image_size = (3, 224, 224)  # ResNet expects 3-channel 224x224 images
seq_length = 10  # Length of the caption sequence

# Random image batch
images = torch.randn(batch_size, *image_size)  # Shape: (batch_size, 3, 224, 224)

# Random captions (word indices)
vocab_size = 30522  # Approximate BERT vocabulary size
captions = torch.randint(0, vocab_size, (batch_size, seq_length))  # Shape: (batch_size, seq_length)

# Forward pass
outputs, alphas = model(images, captions)

# Print output shapes
print("Outputs shape:", outputs.shape)  # Expected: (batch_size, seq_length, vocab_size)
print("Alphas shape:", alphas.shape)    # Expected: (batch_size, seq_length, 49)


C:\Users\austi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\austi\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Outputs shape: torch.Size([2, 10, 30522])
Alphas shape: torch.Size([2, 10, 49])
